In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Attack_Prediction.csv')

In [2]:
columns=df.columns

for column in columns:
    if df[column].dtype=="int32":
        df[column]=df[column].astype("int16")
    elif df[column].dtype=="float64":
        df[column]=df[column].astype("float16")
    elif df[column].dtype=="object":
        df[column]=df[column].astype("category")

In [3]:
df['Sex']= df['Sex'].map({'Female': 0, 'Male': 1})
df['Sex']= pd.to_numeric(df['Sex'])

df['Diet']= df['Diet'].map({'Healthy': 0, 'Average': 1, 'Unhealthy':2})
df['Diet']= pd.to_numeric(df['Diet'])

df[['HBP', 'LBP']]= df['Blood Pressure'].str.split('/', expand= True)
df['HBP']= pd.to_numeric(df['HBP'])
df['LBP']= pd.to_numeric(df['LBP'])

df['Diabetes'] = df['Diabetes'].map({0: 1, 1: 0})

df['Exercise Hours Per Week']= round(df['Exercise Hours Per Week'], 0)

df['Sedentary Hours Per Day']= round(df['Sedentary Hours Per Day'], 0)

df['Income']= round(df['Income'], 0)

df['BMI']= round(df['BMI'], 0)

df = df.drop(columns=['Patient ID', 'Blood Pressure', 'Country', 'Continent', 'Hemisphere'])

In [4]:
X= df.drop(['Heart Attack Risk'], axis= 1)
y= df['Heart Attack Risk']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [6]:
from imblearn.combine import SMOTETomek

smt= SMOTETomek(random_state= 15)
X_sm_tl, y_sm_tl= smt.fit_resample(X_train, y_train)

In [7]:
from sklearn.linear_model import Lasso

lasso= Lasso(alpha= 0.01)
lasso.fit(X_sm_tl, y_sm_tl)

Lasso(alpha=0.01)

In [8]:
selected_features= np.where(lasso.coef_ != 0)[0]
X_sm_tl_selected= X_sm_tl[:, selected_features]
X_sm_tl_test_selected= X_test[:, selected_features]

print(f"Number of features selected: {len(selected_features)}")

Number of features selected: 9


In [9]:
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)
    y_pred_prob= model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None

    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred, zero_division=0))
    print('Precision:', precision_score(y_test, y_pred, zero_division=0))
    print('Accuracy:', accuracy_score(y_test, y_pred))
    if y_pred_prob is not None:
        print('AUC:', roc_auc_score(y_test, y_pred_prob))
    print('-' * 50)

--- LogisticRegression ---

In [10]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(random_state= 15)

train_and_evaluate_model(lr, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[544 581]
 [324 304]]
              precision    recall  f1-score   support

           0       0.63      0.48      0.55      1125
           1       0.34      0.48      0.40       628

    accuracy                           0.48      1753
   macro avg       0.49      0.48      0.47      1753
weighted avg       0.53      0.48      0.49      1753

Precision: 0.34350282485875705
Accuracy: 0.4837421563034798
AUC: 0.4848803963198868
--------------------------------------------------


--- DecisionTreeClassifier ---

In [11]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15)

train_and_evaluate_model(dt, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[697 428]
 [372 256]]
              precision    recall  f1-score   support

           0       0.65      0.62      0.64      1125
           1       0.37      0.41      0.39       628

    accuracy                           0.54      1753
   macro avg       0.51      0.51      0.51      1753
weighted avg       0.55      0.54      0.55      1753

Precision: 0.3742690058479532
Accuracy: 0.5436394751853965
AUC: 0.5135994338287333
--------------------------------------------------


--- Tuned - DecisionTreeClassifier ---

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 5, 'min_samples_split': 2}
Best Precision Score: 0.6250674694419338


In [13]:
tuned_dt= gs_dt.best_estimator_
train_and_evaluate_model(tuned_dt, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[1060   65]
 [ 582   46]]
              precision    recall  f1-score   support

           0       0.65      0.94      0.77      1125
           1       0.41      0.07      0.12       628

    accuracy                           0.63      1753
   macro avg       0.53      0.51      0.45      1753
weighted avg       0.56      0.63      0.54      1753

Precision: 0.4144144144144144
Accuracy: 0.6309184255561894
AUC: 0.5137523000707713
--------------------------------------------------


--- KNeighborsClassifier ---

In [14]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier()
train_and_evaluate_model(knn, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[599 526]
 [318 310]]
              precision    recall  f1-score   support

           0       0.65      0.53      0.59      1125
           1       0.37      0.49      0.42       628

    accuracy                           0.52      1753
   macro avg       0.51      0.51      0.51      1753
weighted avg       0.55      0.52      0.53      1753

Precision: 0.3708133971291866
Accuracy: 0.5185396463205932
AUC: 0.5080842179759377
--------------------------------------------------


--- Tuned - KNeighborsClassifier ---

In [15]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn= GridSearchCV(estimator= knn, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_knn.best_params_)
print("Best Precision Score:", gs_knn.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'uniform'}
Best Precision Score: 0.7348603789509164


In [16]:
tuned_knn= gs_knn.best_estimator_
train_and_evaluate_model(tuned_knn, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[904 221]
 [508 120]]
              precision    recall  f1-score   support

           0       0.64      0.80      0.71      1125
           1       0.35      0.19      0.25       628

    accuracy                           0.58      1753
   macro avg       0.50      0.50      0.48      1753
weighted avg       0.54      0.58      0.55      1753

Precision: 0.3519061583577713
Accuracy: 0.5841414717626925
AUC: 0.4947714083510262
--------------------------------------------------


--- GaussianNB ---

In [17]:
from sklearn.naive_bayes import GaussianNB
nb= GaussianNB()
train_and_evaluate_model(nb, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[576 549]
 [334 294]]
              precision    recall  f1-score   support

           0       0.63      0.51      0.57      1125
           1       0.35      0.47      0.40       628

    accuracy                           0.50      1753
   macro avg       0.49      0.49      0.48      1753
weighted avg       0.53      0.50      0.51      1753

Precision: 0.3487544483985765
Accuracy: 0.49629207073588133
AUC: 0.4795371549893843
--------------------------------------------------


--- SVM ---

In [18]:
from sklearn.svm import SVC
svc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
train_and_evaluate_model(svc, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[767 358]
 [437 191]]
              precision    recall  f1-score   support

           0       0.64      0.68      0.66      1125
           1       0.35      0.30      0.32       628

    accuracy                           0.55      1753
   macro avg       0.49      0.49      0.49      1753
weighted avg       0.53      0.55      0.54      1753

Precision: 0.34790528233151186
Accuracy: 0.5464917284654878
AUC: 0.4942208067940552
--------------------------------------------------


--- Random Forest ---

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state=15)
train_and_evaluate_model(rf, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[955 170]
 [539  89]]
              precision    recall  f1-score   support

           0       0.64      0.85      0.73      1125
           1       0.34      0.14      0.20       628

    accuracy                           0.60      1753
   macro avg       0.49      0.50      0.46      1753
weighted avg       0.53      0.60      0.54      1753

Precision: 0.3436293436293436
Accuracy: 0.5955504848830576
AUC: 0.4935513092710545
--------------------------------------------------


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best Precision Score: 0.8523196063573331


In [21]:
tuned_rf= gs_tuned_rf.best_estimator_
train_and_evaluate_model(tuned_rf, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[959 166]
 [544  84]]
              precision    recall  f1-score   support

           0       0.64      0.85      0.73      1125
           1       0.34      0.13      0.19       628

    accuracy                           0.59      1753
   macro avg       0.49      0.49      0.46      1753
weighted avg       0.53      0.59      0.54      1753

Precision: 0.336
Accuracy: 0.5949800342270394
AUC: 0.5006206652512384
--------------------------------------------------


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state=15)
train_and_evaluate_model(ada, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[794 331]
 [459 169]]
              precision    recall  f1-score   support

           0       0.63      0.71      0.67      1125
           1       0.34      0.27      0.30       628

    accuracy                           0.55      1753
   macro avg       0.49      0.49      0.48      1753
weighted avg       0.53      0.55      0.54      1753

Precision: 0.338
Accuracy: 0.549343981745579
AUC: 0.486617126680821
--------------------------------------------------


In [23]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.6590501578274092


In [24]:
tuned_ada= gs_ada.best_estimator_
train_and_evaluate_model(tuned_ada, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[1082   43]
 [ 605   23]]
              precision    recall  f1-score   support

           0       0.64      0.96      0.77      1125
           1       0.35      0.04      0.07       628

    accuracy                           0.63      1753
   macro avg       0.49      0.50      0.42      1753
weighted avg       0.54      0.63      0.52      1753

Precision: 0.3484848484848485
Accuracy: 0.6303479749001711
AUC: 0.4975859872611465
--------------------------------------------------


--- GradientBoosting ---

In [25]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)
train_and_evaluate_model(grb, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[1106   19]
 [ 614   14]]
              precision    recall  f1-score   support

           0       0.64      0.98      0.78      1125
           1       0.42      0.02      0.04       628

    accuracy                           0.64      1753
   macro avg       0.53      0.50      0.41      1753
weighted avg       0.56      0.64      0.51      1753

Precision: 0.42424242424242425
Accuracy: 0.638904734740445
AUC: 0.48682519462137297
--------------------------------------------------


In [26]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.7257639977763283


In [28]:
tuned_grb= gs_grb.best_estimator_
train_and_evaluate_model(tuned_grb, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[1064   61]
 [ 584   44]]
              precision    recall  f1-score   support

           0       0.65      0.95      0.77      1125
           1       0.42      0.07      0.12       628

    accuracy                           0.63      1753
   macro avg       0.53      0.51      0.44      1753
weighted avg       0.56      0.63      0.54      1753

Precision: 0.41904761904761906
Accuracy: 0.6320593268682259
AUC: 0.4964812455767869
--------------------------------------------------


--- XGB ---

In [29]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)
train_and_evaluate_model(xgb, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[913 212]
 [489 139]]
              precision    recall  f1-score   support

           0       0.65      0.81      0.72      1125
           1       0.40      0.22      0.28       628

    accuracy                           0.60      1753
   macro avg       0.52      0.52      0.50      1753
weighted avg       0.56      0.60      0.57      1753

Precision: 0.396011396011396
Accuracy: 0.6001140901312036
AUC: 0.5214182590233546
--------------------------------------------------


In [30]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_
train_and_evaluate_model(tuned_xgb, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.7257272612827685
[[1027   98]
 [ 561   67]]
              precision    recall  f1-score   support

           0       0.65      0.91      0.76      1125
           1       0.41      0.11      0.17       628

    accuracy                           0.62      1753
   macro avg       0.53      0.51      0.46      1753
weighted avg       0.56      0.62      0.55      1753

Precision: 0.40606060606060607
Accuracy: 0.6240730176839704
AUC: 0.5142675159235669
--------------------------------------------------


--- LGBM ---

In [31]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)
train_and_evaluate_model(lgm, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[LightGBM] [Info] Number of positive: 4288, number of negative: 4288
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000355 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2183
[LightGBM] [Info] Number of data points in the train set: 8576, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[[1016  109]
 [ 569   59]]
              precision    recall  f1-score   support

           0       0.64      0.90      0.75      1125
           1       0.35      0.09      0.15       628

    accuracy                           0.61      1753
   macro avg       0.50      0.50      0.45      1753
weighted avg       0.54      0.61      0.53      1753

Precision: 0.35119047619047616
Accuracy: 0.6132344552196235
AUC: 0.4888181174805379
--------------------------------------------------


In [32]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_
train_and_evaluate_model(tuned_lgm, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 4288, number of negative: 4288
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001242 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4911
[LightGBM] [Info] Number of data points in the train set: 8576, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Parameters: {'learning_rate': 0.01, 'min_data_in_leaf': 30, 'num_leaves': 31, 'reg_alpha': 0.1}
Best Precision Score: 0.8482220142848164
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_le

--- CatBoost ---

In [33]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(learning_rate= 0.1, depth= 6, iterations= 100, random_state= 15, verbose= 0)
train_and_evaluate_model(cat, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

[[1085   40]
 [ 610   18]]
              precision    recall  f1-score   support

           0       0.64      0.96      0.77      1125
           1       0.31      0.03      0.05       628

    accuracy                           0.63      1753
   macro avg       0.48      0.50      0.41      1753
weighted avg       0.52      0.63      0.51      1753

Precision: 0.3103448275862069
Accuracy: 0.6292070735881347
AUC: 0.49413305024769993
--------------------------------------------------


In [34]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_
train_and_evaluate_model(tuned_cat, X_sm_tl_selected, y_sm_tl, X_sm_tl_test_selected, y_test)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best Parameters: {'depth': 10, 'iterations': 100, 'learning_rate': 0.01}
Best Precision Score: 0.820668239883676
[[1080   45]
 [ 600   28]]
              precision    recall  f1-score   support

           0       0.64      0.96      0.77      1125
           1       0.38      0.04      0.08       628

    accuracy                           0.63      1753
   macro avg       0.51      0.50      0.42      1753
weighted avg       0.55      0.63      0.52      1753

Precision: 0.3835616438356164
Accuracy: 0.6320593268682259
AUC: 0.4853191790516631
--------------------------------------------------


In [40]:
y_pred_lr= lr.predict(X_sm_tl_test_selected)
y_pred_prob_lr= lr.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_dt= dt.predict(X_sm_tl_test_selected)
y_pred_prob_dt= dt.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_knn= knn.predict(X_sm_tl_test_selected)
y_pred_prob_knn= knn.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_knn= tuned_knn.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_knn= tuned_knn.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_nb= nb.predict(X_sm_tl_test_selected)
y_pred_prob_nb= nb.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_svc= svc.predict(X_sm_tl_test_selected)
y_pred_prob_svc= svc.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_rf= rf.predict(X_sm_tl_test_selected)
y_pred_prob_rf= rf.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_ada= ada.predict(X_sm_tl_test_selected)
y_pred_prob_ada= ada.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30


--- Visualisation ---

In [41]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0)
                  }

be_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
be_precision= be_precision.sort_values(by= 'Precision Score', ascending=False)
print(be_precision)

                                    Model  Precision Score
11            GradientBoosting Precision:         0.424242
12      Tuned GradientBoosting Precision:         0.419048
2          Tuned Decision Tree Precision:         0.414414
14                   Tuned XGB Precision:         0.406061
16                  Tuned LGBM Precision:         0.400000
13                         XGB Precision:         0.396011
18              Tuned CatBoost Precision:         0.383562
1                Decision Tree Precision:         0.374269
3         KNeighborsClassifier Precision:         0.370813
4   Tuned KNeighborsClassifier Precision:         0.351906
15                        LGBM Precision:         0.351190
5                   GaussianNB Precision:         0.348754
10              Tuned AdaBoost Precision:         0.348485
6                          SVM Precision:         0.347905
7                Random Forest Precision:         0.343629
0          Logistic Regression Precision:         0.3435

In [42]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat)
                  }

be_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
be_accuracy= be_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(be_accuracy)

                                   Model  Accuracy Score
11            GradientBoosting Accuracy:        0.638905
16                  Tuned LGBM Accuracy:        0.637193
18              Tuned CatBoost Accuracy:        0.632059
12      Tuned GradientBoosting Accuracy:        0.632059
2          Tuned Decision Tree Accuracy:        0.630918
10              Tuned AdaBoost Accuracy:        0.630348
17                    CatBoost Accuracy:        0.629207
14                   Tuned XGB Accuracy:        0.624073
15                        LGBM Accuracy:        0.613234
13                         XGB Accuracy:        0.600114
7                Random Forest Accuracy:        0.595550
8          Tuned Random Forest Accuracy:        0.594980
4   Tuned KNeighborsClassifier Accuracy:        0.584141
9                     AdaBoost Accuracy:        0.549344
6                          SVM Accuracy:        0.546492
1                Decision Tree Accuracy:        0.543639
3         KNeighborsClassifier 

In [43]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat)
                  }

be_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
be_auc= be_auc.sort_values(by= 'AUC Score', ascending=False)
print(be_auc)

                              Model  AUC Score
13                         XGB AUC:   0.521418
14                   Tuned XGB AUC:   0.514268
2          Tuned Decision Tree AUC:   0.513752
1                Decision Tree AUC:   0.513599
3         KNeighborsClassifier AUC:   0.508084
8          Tuned Random Forest AUC:   0.500621
10              Tuned AdaBoost AUC:   0.497586
12      Tuned GradientBoosting AUC:   0.496481
4   Tuned KNeighborsClassifier AUC:   0.494771
6                          SVM AUC:   0.494221
17                    CatBoost AUC:   0.494133
7                Random Forest AUC:   0.493551
16                  Tuned LGBM AUC:   0.493150
15                        LGBM AUC:   0.488818
11            GradientBoosting AUC:   0.486825
9                     AdaBoost AUC:   0.486617
18              Tuned CatBoost AUC:   0.485319
0          Logistic Regression AUC:   0.484880
5                   GaussianNB AUC:   0.479537
